In [1]:
import os
import numpy as np
from scipy.signal import find_peaks
from backtesting import Backtest, Strategy
from backtesting.lib import FractionalBacktest


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Data

In [2]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

DB_URL = os.getenv('DB_URL')

engine = create_engine(
    DB_URL,
    pool_pre_ping=True,   # checks connection before using
    pool_recycle=1800,    # optional: avoids stale timeouts
    connect_args={"check_same_thread": False} if "sqlite" in DB_URL else {},
)
# engine

In [3]:
import pandas as pd
from datetime import datetime
SCHEMA = 'proddb.'
tables = {
    'p5m': SCHEMA+'coin_prices_5m',
    'p1h': SCHEMA+'coin_prices_1h',
    'f5m': SCHEMA+'f_coin_signal_5m',
    'f10m': SCHEMA+'f_coin_signal_10m',
    'f15m': SCHEMA+'f_coin_signal_15m',
    'f30m': SCHEMA+'f_coin_signal_30m',
    'f1h': SCHEMA+'f_coin_signal_1h',
    'f4h': SCHEMA+'f_coin_signal_4h',
    'f1d': SCHEMA+'f_coin_signal_1d',
    'f1D': SCHEMA+'f_coin_signal_1d',
    'orders': SCHEMA+'trade_orders_sim',
    'tp_by_sess': SCHEMA+'trade_orders_tp_by_session',
    }

from_time = int(datetime(2025, 4, 1, 0, 0, 0).timestamp())
to_time = int(datetime(2025, 9, 1, 0, 0, 0).timestamp())
from_time, to_time
def get_data_4_8(symbol, tf='f1h', extr_cols=[]):
    try:
        table = tables[tf]
    except Exception as e:
        print(e)
        return tables['p1h']
    
    if len(extr_cols) > 0:
        extr_cols_str = ', ' + ', '.join(extr_cols)
    else:
        extr_cols_str = ''

    df = pd.read_sql(f"""
            select TO_TIMESTAMP(open_time) as open_time, open as Open, close as Close,
                high  as High, low as Low, volume as Volume 
                {extr_cols_str}
            from {table} 
            where open_time >= {from_time} and open_time < {to_time}
                and symbol = '{symbol}'
            order by open_time asc
        """,
        engine, 
        index_col="open_time")
    df.rename(columns={
        'open': 'Open',
        'close': 'Close',
        'high': 'High',
        'low': 'Low',
        'volume': 'Volume',
        'rsi7': 'rsi'
    }, inplace=True)
    return df
df = get_data_4_8('HBARUSDT', 'f1h', ['rsi7', 'rsi14', 'adx'])

In [5]:
import requests

# 0xd4e31cb3c21831503b3d699e595363aa7f807104

# # "https://api.geckoterminal.com/api/v2/networks/hedera-hashgraph/pools/0xb4e267d955b0bbbc1ba5f39f9c92cc8369a1f712/ohlcv/minute?aggregate=5&before_timestamp=1751352000&limit=5&include_empty_intervals=true"
# "https://api.geckoterminal.com/api/v2/networks/hedera-hashgraph/pools/0xb4e267d955b0bbbc1ba5f39f9c92cc8369a1f712/ohlcv/minute?aggregate=5&before_timestamp=1751352000&limit=5&include_empty_intervals=true"


In [ ]:
coingecko_url = "https://api.geckoterminal.com/api/v2/networks/hedera-hashgraph/pools/{pool_id}/ohlcv/{timeframe}?aggregate={aggregate}&before_timestamp={before_timestamp}&limit={limit}&include_empty_intervals=true"


end_time = int(datetime.now().timestamp())
url = coingecko_url.format(
    pool_id='0xb7cdd448e06c5cc82e12e8fe1d124281b2c4c3c3',  # WETH/USDC v1
    timeframe = 'hour',
    aggregate=1,
    before_timestamp=end_time,
    limit=5,
    )
response = requests.get(url)
end_time, response.json()

(1758876711,
 {'data': {'id': '72d1885f-4ea5-4a6a-a7b3-58eb2cff27ca',
   'type': 'ohlcv_request_response',
   'attributes': {'ohlcv_list': [[1758798000,
      3565.57838614367,
      3565.57838614367,
      3537.53117582805,
      3537.53117582805,
      0.00247627182307963],
     [1758794400,
      3502.97648728935,
      3565.57838614367,
      3502.97648728935,
      3565.57838614367,
      0.00249590487030057],
     [1758790800,
      3502.97648728935,
      3502.97648728935,
      3502.97648728935,
      3502.97648728935,
      0.0],
     [1758787200,
      3502.97648728935,
      3502.97648728935,
      3502.97648728935,
      3502.97648728935,
      0.0],
     [1758783600,
      3502.97648728935,
      3502.97648728935,
      3502.97648728935,
      3502.97648728935,
      0.0]]}},
  'meta': {'base': {'address': '0x000000000000000000000000000000000008437c',
    'name': 'Wrapped Ether[hts]',
    'symbol': 'WETH[hts]',
    'coingecko_coin_id': None},
   'quote': {'address': '0x000

In [ ]:
end_time = int(datetime.now().timestamp())
url = coingecko_url.format(
    pool_id='0x8811f36a4bd217ca3e2334522ba16dc3dc127174',  # WETH/WHBAR v1
    timeframe = 'hour',
    aggregate=1,
    before_timestamp=end_time-3600*24*30,
    limit=5,
    )
response = requests.get(url)
end_time, response.json()

{'data': {'id': 'a0d4a404-184a-449a-a3e2-daa1705d086e',
  'type': 'ohlcv_request_response',
  'attributes': {'ohlcv_list': [[1758801000,
     3565.57838614367,
     3565.57838614367,
     3537.53117582805,
     3537.53117582805,
     0.00247627182307963],
    [1758800700,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     0.0],
    [1758800400,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     0.0],
    [1758800100,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     0.0],
    [1758799800,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     3565.57838614367,
     0.0]]}},
 'meta': {'base': {'address': '0x000000000000000000000000000000000008437c',
   'name': 'Wrapped Ether[hts]',
   'symbol': 'WETH[hts]',
   'coingecko_coin_id': None},
  'quote': {'address': '0x000000000000000000000000000000000006f89a',
   'name': 'USD Coin',
   

In [28]:
import json 

In [31]:
a={
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5,
    'f': 6,
    'g': 7,
}

json.dumps(a)

'{"a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6, "g": 7}'

In [33]:
amount_in = 100
best_paths = [a]
line = f"time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, amount_in: {amount_in}, best_paths: {json.dumps(best_paths)}"
open('best_paths.log', 'a').write(line)


113